In [4]:
#install packages
!pip install faiss-cpu
#!pip install sentence transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement sentence (from versions: none)
ERROR: No matching distribution found for sentence


In [5]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------  8.7/8.7 MB 48.8 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 41.7 MB/s  0:00:00

   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   ------------- -------------------------- 1/3 [scikit-learn]
   -----

In [6]:
import pandas as pd

pd.set_option('display.max_colwidth',100)

In [7]:
df = pd.read_csv("faiss.csv")
df.shape

(10, 2)

In [8]:
df

,text,category
0,Regular exercise and a balanced diet are essential for maintaining good health,health
1,The stock market reached record highs this quarter due to strong corporate earnings,finance
2,The home team won the championship after an intense overtime period,sports
3,Artificial intelligence is revolutionizing how we interact with digital devices.,technology
4,Patients should consult their doctor before starting any new medication,health
5,Investors are diversifying their portfolios to manage risk in volatile markets,finance
6,he Olympic athletes trained for years to compete at the international level.,sports
7,Blockchain technology has the potential to transform financial transactions,technology
8,Getting adequate sleep is crucial for both physical and mental well-being,health
9,any companies are adopting cloud computing to improve efficiency and scalabilit,technology


In [10]:
from sentence_transformers import SentenceTransformer

In [28]:
df.text

0         Regular exercise and a balanced diet are essential for maintaining good health
1    The stock market reached record highs this quarter due to strong corporate earnings
2                    The home team won the championship after an intense overtime period
3       Artificial intelligence is revolutionizing how we interact with digital devices.
4                Patients should consult their doctor before starting any new medication
5         Investors are diversifying their portfolios to manage risk in volatile markets
6           he Olympic athletes trained for years to compete at the international level.
7            Blockchain technology has the potential to transform financial transactions
8              Getting adequate sleep is crucial for both physical and mental well-being
9        any companies are adopting cloud computing to improve efficiency and scalabilit
Name: text, dtype: object

In [13]:
encoder_model = SentenceTransformer("all-mpnet-base-v2")
encoder_model.encode(df.text)
vectors = encoder_model.encode(df.text)
vectors.shape


(10, 768)

In [14]:
dim = vectors.shape[1] #to get 768 
dim

768

In [ ]:
# setting up our vector database FAISS
#TO create instance of that databse : index of that database we need 768 

In [15]:
import faiss

index = faiss.IndexFlatL2(dim)
index.add(vectors)

In [16]:
search_query = "I am looking for updates on current political conditons"
search_vector = encoder_model.encode(search_query)
search_vector.shape

(768,)

In [17]:
# we need to make this 2d array currently it is 1d array 
import numpy as np
search_vector = np.array(search_vector).reshape(1,-1)
search_vector.shape

(1, 768)

In [18]:
index.search(search_vector,k=2) #this finction need 2d array, k = how many similar vectors 

(array([[1.6835442, 1.7231779]], dtype=float32), array([[1, 9]]))

In [24]:
# it is giving a tuple 
#so lets store it in a way
distance, I = index.search(search_vector,k=2)
I

array([[1, 9]])

In [23]:
df.loc[[I[0][0],I[0][1]]]

,text,category
1,The stock market reached record highs this quarter due to strong corporate earnings,finance
9,any companies are adopting cloud computing to improve efficiency and scalabilit,technology


In [ ]:
#lets put it all togethe just sentence and search results

In [27]:
search_query = "medicine"
search_vector = encoder_model.encode(search_query)
search_vector = np.array(search_vector).reshape(1,-1)
distance, I = index.search(search_vector,k=2)
df.loc[[I[0][0],I[0][1]]]

,text,category
4,Patients should consult their doctor before starting any new medication,health
0,Regular exercise and a balanced diet are essential for maintaining good health,health
